In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('./datasets/train.csv', index_col='id')
test = pd.read_csv('./datasets/test.csv', index_col='id')

df = pd.concat([train, test])

train['type'].value_counts()

Ghoul     129
Goblin    125
Ghost     117
Name: type, dtype: int64

In [3]:
df.drop('color', axis=1, inplace=True)
df.head()

,bone_length,rotting_flesh,hair_length,has_soul,type
id,,,,,
0,0.354512,0.350839,0.465761,0.781142,Ghoul
1,0.575560,0.425868,0.531401,0.439899,Goblin
2,0.467875,0.354330,0.811616,0.791225,Ghoul
4,0.776652,0.508723,0.636766,0.884464,Ghoul
5,0.566117,0.875862,0.418594,0.636438,Ghost


In [4]:
df.describe()

,bone_length,rotting_flesh,hair_length,has_soul
count,900.000000,900.000000,900.000000,900.000000
mean,0.429054,0.504961,0.522163,0.467104
std,0.134744,0.153567,0.170981,0.178150
min,0.000000,0.000000,0.000000,0.000000
25%,0.332085,0.402419,0.396112,0.343909
50%,0.426781,0.505279,0.530294,0.465508
75%,0.518167,0.605218,0.644958,0.589235
max,1.000000,1.000000,1.000000,1.000000


In [5]:
columns = df.drop('type', axis=1).columns

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])
types = le.classes_

In [7]:
# means 
for col in columns:
    print(col)
    sorted_col_index = all_df[col].sort_values().index
    for typ in types:
        if len(np.where(types == typ)[0]) > 0:
            type_mask = (all_df['type'] == np.where(types == typ)[0][0])
            print(typ)
            print(all_df.loc[type_mask, col].mean())
    fig, ax = plt.subplots()   
    ax.scatter(all_df.loc[sorted_col_index, col], all_df.loc[sorted_col_index, 'type'])
    ax.set_title(col)
    print()

bone_length


NameError: name 'all_df' is not defined

In [8]:
bone_var = 0.35

bone_ghost_mean = 0.35
bone_ghoul_mean = 0.53
bone_goblin_mean = 0.43
bone_means = [bone_ghost_mean, bone_ghoul_mean, bone_goblin_mean]

hair_var = 0.37

hair_ghost_mean = 0.37
hair_ghoul_mean = 0.66
hair_goblin_mean = 0.55
hair_means = [hair_ghost_mean, hair_ghoul_mean, hair_goblin_mean]

soul_var = 0.32

soul_ghost_mean = 0.32
soul_ghoul_mean = 0.6
soul_goblin_mean = 0.47
soul_means = [soul_ghost_mean, soul_ghoul_mean, soul_goblin_mean]

flesh_var = 0.44

flesh_ghost_mean = 0.59
flesh_ghoul_mean = 0.49
flesh_goblin_mean = 0.44
flesh_means = [flesh_ghost_mean, flesh_ghoul_mean, flesh_goblin_mean]

means = np.array([bone_means, flesh_means, hair_means, soul_means])
var_s = np.array([bone_var, flesh_var, hair_var, soul_var]) * 1.1

In [9]:
df.columns

Index(['bone_length', 'rotting_flesh', 'hair_length', 'has_soul', 'type'], dtype='object')

In [10]:
means

array([[0.35, 0.53, 0.43],
       [0.59, 0.49, 0.44],
       [0.37, 0.66, 0.55],
       [0.32, 0.6 , 0.47]])

In [11]:
var_s

array([0.385, 0.484, 0.407, 0.352])

In [12]:
vote_table = np.zeros((df.shape[0], 3))

In [13]:
cols = df.drop('type', axis=1).columns
typs = [0, 1, 2]
for i in range(df.shape[0]):
    creature = df.iloc[i]
    if creature['type'] == 3:
        for j in range(len(cols)):
            for typ in typs:
                if creature[cols[j]] >= means[j, typ] - var_s[j] and creature[cols[j]] <= means[j, typ] + var_s[j]:
                    vote_table[i, typ] += 1
        ind = np.argmax(vote_table[i])
        if vote_table[i].max() > 3 and np.where(vote_table[i] == vote_table[i, ind])[0].shape[0] == 1:
            df.iloc[i, -1] = ind
#     if df.iloc[i, 'type']

In [14]:
df['type'].value_counts()

3    498
1    142
0    132
2    128
Name: type, dtype: int64

In [15]:
test_df = df[(df['type'] == 3)].drop('type', axis=1)
train_df = df[(df['type'] != 3)]
y_train = train_df['type']

train_df.drop('type', axis=1, inplace=True)
train_df.head()

c:\users\kmbs\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,bone_length,rotting_flesh,hair_length,has_soul
id,,,,
0,0.354512,0.350839,0.465761,0.781142
1,0.575560,0.425868,0.531401,0.439899
2,0.467875,0.354330,0.811616,0.791225
4,0.776652,0.508723,0.636766,0.884464
5,0.566117,0.875862,0.418594,0.636438


In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
model = LogisticRegression()
model.fit(train_df, y_train)

LogisticRegression()

In [18]:
y_pred = model.predict(test_df)

In [19]:
test_df['type'] = y_pred

In [20]:
train_df['type'] = y_train

c:\users\kmbs\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
all_df = pd.concat([train_df, test_df])

In [22]:
submission_df = pd.read_csv('./datasets/sample_submission.csv', index_col='id')
prediction_types = types[all_df.loc[submission_df.index, 'type']]
submission_df['type'] = prediction_types

submission_df.to_csv('./results/LR.csv')

In [23]:
all_df['type'].value_counts()

1    335
0    308
2    257
Name: type, dtype: int64

# Hair

In [ ]:
bone_ind = df['bone_length'].sort_values().index

In [ ]:
plt.scatter(df.loc[bone_ind].index, df.loc[bone_ind]['rotting_flesh'])

In [ ]:
df.loc[bone_ind, 'type'].head(225).value_counts()

In [ ]:
min_Ghost_bone_length = df.loc[bone_ind, 'bone_length'].iloc[0]
mean_Ghost_bone_length = df.loc[bone_ind, 'bone_length'].iloc[225]
print('bone_length mean for Ghost')
print(mean_Ghost_bone_length)
print('varience =', mean_Ghost_bone_length - min_Ghost_bone_length)

In [ ]:
bone_Ghost_mask = df['bone_length'] <= 2*mean_Ghost_bone_length

In [ ]:
df.loc[bone_Ghost_mask, 'type'].value_counts()

# Hair

In [ ]:
hair_ind = df['hair_length'].sort_values().index

In [ ]:
df.loc[hair_ind, 'type'].head(50).value_counts()

In [ ]:
df.loc[bone_ind, 'type'].head(40).value_counts()

# Soul

In [ ]:
soul_ind = df['has_soul'].sort_values().index

In [ ]:
df.loc[soul_ind, 'type'].head(50).value_counts()

# Flesh

In [ ]:
flesh_ind = df['rotting_flesh'].sort_values().index

In [ ]:
df.loc[flesh_ind, 'type'].head(50).value_counts()

# REVERSE

# Bone

In [ ]:
rev_bone_ind = df['bone_length'].sort_values(ascending=False).index

In [ ]:
df.loc[rev_bone_ind, 'type'].head(15).value_counts()

# Hair

In [ ]:
rev_hair_ind = df['hair_length'].sort_values(ascending=False).index

In [ ]:
df.loc[rev_hair_ind, 'type'].head(20).value_counts()

# Soul

In [ ]:
rev_soul_ind = df['has_soul'].sort_values(ascending=False).index

In [ ]:
df.loc[rev_soul_ind, 'type'].head(10).value_counts()

# Flesh

In [ ]:
rev_flesh_ind = df['rotting_flesh'].sort_values(ascending=False).index

In [ ]:
df.loc[rev_flesh_ind, 'type'].head(10).value_counts()

# Fill types

In [ ]:
df.loc[bone_ind[:35], 'type'] = 'Ghost'
df.loc[hair_ind[:40], 'type'] = 'Ghost'
df.loc[soul_ind[:50], 'type'] = 'Ghost'
df.loc[rev_bone_ind[:15], 'type'] = 'Ghoul'
df.loc[rev_hair_ind[:20], 'type'] = 'Ghoul'
df.loc[rev_soul_ind[:10], 'type'] = 'Ghoul'
df.loc[rev_flesh_ind[:10], 'type'] = 'Ghost'

In [ ]:
df['type'].value_counts()

# New train sample

In [ ]:
train_df = df.loc[df['type'].notna()]

# Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

y = le.fit_transform(train_df['type'])
types = le.classes_

In [ ]:
train_df.drop('type'], axis=1, inplace=True)

In [ ]:
train_df.head()

In [ ]:
test.head()

# Training simple model: logreg

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression()
clf.fit(train_df, y)

In [ ]:
predictions = clf.predict(test_df)

In [ ]:
submission_df = pd.read_csv('./datasets/sample_submission.csv', index_col='id')
submission_df['type'] = types[predictions]

submission_df.to_csv('./results/LogReg_NEW.csv')

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=1, random_state=2)
model.fit(train_df, y)
predictions = model.predict(test)

In [ ]:
submission_df = pd.read_csv('./datasets/sample_submission.csv', index_col='id')
submission_df['type'] = types[predictions]

submission_df.to_csv('./results/RanFor_NEW.csv')